In [1]:
import os
import random
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from skimage.color import gray2rgb, rgba2rgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

name_mapping = {
    "cane": "dog",
    "gatto": "cat",
    "cavallo": "horse",
    "elefante": "elephant",
    "farfalla": "butterfly",
    "gallina": "chicken",
    "mucca": "cow",
    "pecora": "sheep",
    "ragno": "spider",
    "scoiattolo": "squirrel",
}

print(len(name_mapping))

# Define the path to the dataset
dataset_path = './raw-img/'

# Define the target size for resizing images
image_size = (64, 64)

# Specify the number of samples per category
samples_per_category = 5000

# Initialize lists to hold the data and labels
data = []
labels = []

# Create reverse mapping for labels
label_map = {v: idx for idx, v in enumerate(name_mapping.values())}
categories = list(name_mapping.values())

# Load the images from each category folder
for italian_name, english_name in name_mapping.items():
    category_path = os.path.join(dataset_path, italian_name)
    image_files = os.listdir(category_path)
    sampled_files = random.sample(image_files, min(samples_per_category, len(image_files)))
    print(english_name)
    for img_name in sampled_files:
        img_path = os.path.join(category_path, img_name)
        try:
            img = imread(img_path)
            # Ensure all images are in RGB format
            if img.shape[-1] == 4:  # RGBA image
                img = rgba2rgb(img)
            elif len(img.shape) == 2:  # Grayscale image
                img = gray2rgb(img)
            img_resized = resize(img, image_size, anti_aliasing=True)
            if img_resized.shape == (64, 64, 3):
                data.append(img_resized)
                labels.append(label_map[english_name])
            else:
                print(f"Skipping image {img_name} due to unexpected shape {img_resized.shape}")
        except Exception as e:
            print(f"Error loading image {img_name}: {e}")
    print(english_name, 'done')

10
dog
dog done
cat
cat done
horse
horse done
elephant
elephant done
butterfly
butterfly done
chicken
chicken done
cow
cow done
sheep
sheep done
spider
spider done
squirrel
squirrel done


In [2]:
# Convert lists to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Normalize the data
data = data / 255.0

# Flatten the images for scikit-learn
data = data.reshape((data.shape[0], -1))

In [3]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42 ,stratify=labels)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [4]:
# Train a Model
svm = SVC(kernel='linear', C=1.0, random_state=42)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

# Evaluate the Model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred, target_names=categories)

print('Confusion Matrix:')
print(conf_matrix)
print('\nClassification Report:')
print(class_report)

Accuracy: 30.54%
Confusion Matrix:
[[335 108  69  57  65  88  33  69 100  49]
 [101  59  15  18  21  29   6   8  46  31]
 [ 73  30 199  44  18  35  52  29  32  13]
 [ 67  16  38  67  15  21  14  15  28   8]
 [ 86  37  19  24  96  39  13  27  56  25]
 [114  57  47  26  31 210  26  32  44  33]
 [ 55  16  69  22  16  24  96  47  22   6]
 [ 68  21  27  16  18  31  26 116  29  12]
 [216  95  37  36  78  57  17  34 344  50]
 [ 79  35  17  20  26  29   6  23  60  77]]

Classification Report:
              precision    recall  f1-score   support

         dog       0.28      0.34      0.31       973
         cat       0.12      0.18      0.15       334
       horse       0.37      0.38      0.37       525
    elephant       0.20      0.23      0.22       289
   butterfly       0.25      0.23      0.24       422
     chicken       0.37      0.34      0.36       620
         cow       0.33      0.26      0.29       373
       sheep       0.29      0.32      0.30       364
      spider       0.45

In [5]:
# Hyperparameter Tuning
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=3, cv=5)
grid.fit(X_train, y_train)

grid_predictions = grid.predict(X_test)

grid_accuracy = accuracy_score(y_test, grid_predictions)
print(f'Grid Search Accuracy: {grid_accuracy * 100:.2f}%')
print('Best Parameters:', grid.best_params_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END C=0.1, gamma=scale, kernel=linear;, score=0.292 total time=106.5min
